# Introduction

Welcome to the P1 Workspace with Group 14:
Will Mitchell
Trevor Gordon


Below are some prelimnary Helper methods that benifit the program as a whole with the starting code parameters

In [ ]:
# Parameters
GROUP_ID = 'Group14'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = '/content/puzzles/Easy-P1.txt'

In [ ]:
def parse_matrix(file_path : str):
  with open(file_path, mode="r") as file:
      new_matrix = []
      for line in file:
        new_line_array = line.replace("\n","").replace("\ufeff","").split(",")
        new_matrix.append(new_line_array)


  file.close()
  return new_matrix

parse_matrix(PUZZLE_PATH)



[['?', '?', '8', '?', '5', '6', '?', '?', '?'],
 ['7', '?', '4', '?', '?', '?', '6', '1', '9'],
 ['?', '?', '?', '?', '?', '?', '8', '5', '?'],
 ['6', '?', '7', '?', '2', '9', '5', '?', '?'],
 ['?', '?', '9', '?', '6', '?', '1', '?', '?'],
 ['?', '?', '2', '3', '1', '?', '9', '?', '4'],
 ['?', '3', '5', '?', '?', '?', '?', '?', '?'],
 ['4', '2', '1', '?', '?', '?', '3', '?', '6'],
 ['?', '?', '?', '8', '3', '?', '4', '?', '?']]

In [ ]:
def get_square_values(sudoku_state, cord_x, cord_y):
  #first box
  if cord_x <= 2 and cord_y <= 2:

  elif (2 < cord_x < 5) and (cord_y <= 2):






  print(answer)


yes


In [ ]:
matrix = parse_matrix(PUZZLE_PATH)
if 6 in matrix[0]:
  return True
else:
  return False


SyntaxError: 'return' outside function (ipython-input-2433683405.py, line 3)

# Helper Functions


In [ ]:
def get_row_values(sudoku_state, cord_x, cord_y):
  matrix = parse_matrix(PUZZLE_PATH)
  value = matrix[cord_x][cord_y]
  my_set = set(matrix[cord_x])
  my_set.remove(value)
  return my_set

print(get_row_values(0,1))

{'8', '?', '6', '5'}


In [ ]:
def get_col_values(sudoku_state, cord_x, cord_y):
  matrix = parse_matrix(PUZZLE_PATH)
  value = matrix[cord_x][cord_y]
  my_set = set(matrix[cord_x])
  my_set.remove(value)
  return my_set

print(get_row_values(0,1))